In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

if not os.path.exists("coins.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/coins.png 
if not os.path.exists("rice.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/rice.png 
if not os.path.exists("catalogue.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/catalogue.png 
if not os.path.exists("bart.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/bart.png 
if not os.path.exists("figura1.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/figura1.png 
if not os.path.exists("figura2.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/figura2.png 
if not os.path.exists("figura3.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/figura3.png 
if not os.path.exists("figura4.png") :
    !wget --no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/figura4.png 

def showImageAndHist(img_name):
  img = cv2.imread(img_name,cv2.IMREAD_GRAYSCALE)
  hist = cv2.calcHist([img],[0],None,[256],[0,256])
  
  fig,axs = plt.subplots(1,2)
 
  fig.suptitle("It is for: "+img_name,fontsize=16)
  axs[0].imshow(img, 'gray', vmin=0, vmax=256)
  axs[0].set_title(img_name)
  axs[0].axis('off')
  axs[1].plot(hist)
  axs[1].set_title("Histogram")
  axs[1].grid()
  plt.rcParams["figure.figsize"] = (10,5)
  plt.xticks(np.arange(0, 256, 20.0))
  plt.show()
  
  return img,hist


def drawBinary(img,threshold,style):
  ret, bw_img = cv2.threshold(img,threshold,255,style)
  plt.imshow(bw_img)
  plt.show()

# coins
coins,histOfCoins = showImageAndHist('coins.png')
drawBinary(coins,80,cv2.THRESH_BINARY)

# figura1
figura1,histOfFigura1 = showImageAndHist('figura1.png')
drawBinary(figura1,80,cv2.THRESH_BINARY)

# figura2
figura2,histOfFigura2 = showImageAndHist('figura2.png')
drawBinary(figura2,130,cv2.THRESH_BINARY)

# figura3
figura3,histOfFigura3 = showImageAndHist('figura3.png')
drawBinary(figura3,100,cv2.THRESH_BINARY)

# figura4
figura4,histOfFigura4 = showImageAndHist('figura4.png')
drawBinary(figura4,50,cv2.THRESH_BINARY)


# -----iteracyjnie-----




def iter_thresholding(img,hist):


  normalized = hist / sum(hist)
  normalized = np.squeeze(normalized)


  histCumSum = normalized.cumsum()
  histCumSumN = histCumSum/histCumSum.max()


  Q = np.arange(len(hist))
  k = int(sum(normalized*Q))
  # print(k)

  while True:
    
    p0 = histCumSumN[k]
    p1 = 1-p0
    
    m0 = 0
    for i in range(k+1):
      m0+=normalized[i]*Q[i]
    m0 = m0/p0

    m1 = 0
    for i in range(k+1,len(hist)):
      m1+=normalized[i]*Q[i]
    m1 = m1/p1

    new_k = (m0+m1)/2

    if abs(new_k - k) < 1:
      break;
    k=int(new_k)

  return k


 




  


thr = iter_thresholding(coins,histOfCoins)
print(thr)
drawBinary(coins,thr,cv2.THRESH_BINARY)

thr = iter_thresholding(figura1,histOfFigura1)
drawBinary(figura1,thr,cv2.THRESH_BINARY)

thr = iter_thresholding(figura2,histOfFigura2)
drawBinary(figura2,thr,cv2.THRESH_BINARY)

thr = iter_thresholding(figura3,histOfFigura3)
drawBinary(figura3,thr,cv2.THRESH_BINARY)

thr = iter_thresholding(figura4,histOfFigura4)
drawBinary(figura4,thr,cv2.THRESH_BINARY)

# ----otsu-----

def otsu(img,hist):
  
  normalized = hist.ravel()/hist.max()

  Q = normalized.cumsum()
  bins = np.arange(256)

  thr=-1
  fn_min = np.inf

  for i in range(256):
    w0,w1 = np.hsplit(bins,[i])
    p0,p1 = np.hsplit(normalized,[i])
    q0=Q[i]
    q1=Q[255]-Q[i] 

    m0 = np.sum(p0*w0)/q0
    m1 = np.sum(p1*w1)/q1

    v0 = np.sum(((w0-m0)**2)*p0)/q0
    v1 = np.sum(((w1-m1)**2)*p1)/q1

    new_fn = v0*q0 + v1*q1
    if new_fn < fn_min:
      fn_min = new_fn
      thr = i

  return thr




thr = otsu(coins,histOfCoins)
print("Otsu by me:",thr)
otsu_thr, ret = cv2.threshold(coins,0,255,cv2.THRESH_OTSU)
print("Otsu by cv2:",otsu_thr)

drawBinary(coins,otsu_thr,cv2.THRESH_BINARY)

rice, histOfRice = showImageAndHist('rice.png')
thr = otsu(rice,histOfRice)
drawBinary(rice,thr,cv2.THRESH_BINARY)


catalogue, histOfCatalogue = showImageAndHist('catalogue.png')
thr = otsu(catalogue,histOfCatalogue)
drawBinary(catalogue,thr,cv2.THRESH_BINARY)



